In [69]:
from dotenv import load_dotenv
load_dotenv()

True

In [70]:
from service.llm_service import LlmFactory

llm = LlmFactory.get_model("llama-3.2-90b-text-preview")

In [71]:
from service.history_service import ChatHistoryFactory, MemoryType

machine_template = (
    """You are an expert at answering user questions about specefic informatioin of capping machines. you will answer questions that asks about :
   - Specific machine details such as:
     * Production speeds and capacity (e.g., bottles per minute)
     * Supported cap types (e.g., pre-threaded plastic caps)
     * Machine versions (standard, washable, inox external surfaces)
     * Technical features (closure heads, caps application)
     * Optional components (caps chute, sorter, caps elevator)
     * Market applications for specific machines
     * Format change capabilities
     * Machine configurations and versions
     * Specific machine models (like EURO PK)
     * Operational features (washing systems, lubrication, etc.)
     * Component specifications (caps chute, pick & place, etc.)
     if you don't know something just admit that you don't know it.
     
     Examples:
    - "What's the maximum speed of the EURO PK capper?"
    - "What types of closure heads are available for the EURO PK?"
    - "Does the EURO PK have a washable version?" → machines_catalog
     """
)
arol_template = (
    """You are an expert at answering user questions about AROL company-wide information.
   - you will answer Question that Asks about Arol at a company-wide level:
     * Overall product portfolio and machine categories
     * Arol machine productions
     * Company history and background
     * Total number of machine types or models
     * Geographic presence and markets served
     * Company-wide capabilities and expertise
     * General industry sectors served across all products
     * Company certifications and standards
     * Overall manufacturing capabilities
     * Business partnerships and relationships
     * Company milestones and achievements
     if you don't know something just admit that you don't know it.
     
     Examples:
    - "How many different capping machine models does Arol produce?"
    - "How many machines do you produce?"
    - "What are Arol's main market segments across all products?"
    - "What's Arol's expertise in the beverage industry?"
     """
)
history_session = ChatHistoryFactory.get_chat_history(memory_type=MemoryType.REDIS)


def create_chat_hisotry(query):
    messages = history_session("amirali049f4d9986f24d49b86003205f5749e6").messages
    formatted_history = []
    #last_chats = messages[-2:]
    for message in messages:
        if message.__class__.__name__ == 'HumanMessage':
            formatted_history.append(f"Human asked: \"{message.content}\"")
        elif message.__class__.__name__ == 'AIMessage':
            formatted_history.append(f"AI Response: {message.content}")

    final_text = "\n\n".join(formatted_history)
    include_last_question = final_text + f"\n\n Human asked: {query["input"]}\""
    return include_last_question

In [72]:
from langchain_community.embeddings import HuggingFaceEmbeddings

model_name = "thenlper/gte-large"  # model_name: The name of the model to use
model_kwargs = {'device': 'cpu'}  # model_kwargs: Keyword arguments to pass to the model
encode_kwargs = {'normalize_embeddings': True}  # encode_kwargs: Keyword arguments to pass to the encode method
embeddings = HuggingFaceEmbeddings(  # HuggingFaceEmbeddings: A class for using Hugging Face models
        model_name=model_name,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs
)

In [73]:
from langchain_community.utils.math import cosine_similarity


prompt_templates = [arol_template, machine_template]
prompt_embeddings = embeddings.embed_documents(prompt_templates)


def prompt_router(input):
    query_embedding = embeddings.embed_query(input["query"])
    similarity = cosine_similarity([query_embedding], prompt_embeddings)[0]
    most_similar = prompt_templates[similarity.argmax()]
    return {"datasource": "arol_information" if most_similar == arol_template else "machine_information"}

In [74]:
input = create_chat_hisotry({"input": "give me information about euro vp"})
prompt_router({"query": input})

{'datasource': 'machine_information'}

In [75]:
print(input)

Human asked: "how many type of machines arol have?"

AI Response: Based on the provided information, AROL appears to have the following types of machines:

1. Cappers:
	* Euro VA-CB (capper for aluminum caps on can bottles)
	* Euro VPA (capper for ROPP aluminum caps with plastic insert)
	* Euro VP (capper for pre-threaded plastic, irregular geometry, and large caps)
	* Equatorque (capper with brushless motors on vertical and rotary spindle control)
	* Euro VA (capper for ROPP aluminum caps)
	* Next VP (pre-threaded plastic caps lube-free capper)
	* Eagle VP (single-head capper for pre-threaded plastic, irregular geometry, and large caps)
	* Eagle VA (single-head capper for ROPP aluminum caps)
2. Corkers:
	* Saturno R (corker for natural, agglomerated, and synthetic corks)
	* Saturno F (corker for natural, agglomerated, and synthetic corks)
	* Gemini R (single-head corker for natural, agglomerated, and synthetic corks)
	* Gemini F (single-head corker for mushroom corks)
3. Pluggers:
	* 